In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from tqdm.auto import tqdm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch_geometric.nn import GATv2Conv, global_mean_pool
import torch.nn as nn
from torch_geometric.nn import GINEConv
import logging
import os
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datetime import datetime
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.metrics import (
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
    roc_curve,
)
import seaborn as sns
import time
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
import warnings

warnings.filterwarnings(
    "ignore"
)  # "error", "ignore", "always", "default", "module" or "once"

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [2]:
graphs = {}
datasets = set()
for p in Path("data/tabular/").glob("*.csv"):
    datasets.add(str(p).split("/")[-1].split(".")[0])

all_data = {}
for dataset in tqdm(datasets):
    all_data[dataset] = {"train": [], "val": [], "test": []}
    graph_data = pd.read_csv(f"data/tabular/{dataset}.graph.csv")
    node_features_data = pd.read_csv(
        f"data/tabular/{dataset}.node_features.csv", index_col=0
    )
    n_nodes = int(graph_data["p2"].iloc[-1].split("_")[-1]) + 1
    n_graphs = int(graph_data.columns[-1]) + 1
    train, test = train_test_split(
        np.arange(n_graphs),
        train_size=0.8,
        stratify=node_features_data["target"].to_numpy(),
    )
    for k in range(n_graphs):
        x = []
        y = []
        edge_index = []
        edge_attr = []
        for r in range(len(graph_data)):
            i = int(graph_data["p1"].iloc[r].split("_")[-1])
            j = int(graph_data["p2"].iloc[r].split("_")[-1])
            v = graph_data[str(k)].iloc[r]
            edge_index.append((i, j))
            edge_attr.append((v,))
        x.append(node_features_data.iloc[k].to_numpy()[:-1])
        y = bool(node_features_data.iloc[k].to_numpy()[-1])
        data = Data(
            x=torch.Tensor(x).T,
            edge_index=torch.LongTensor(edge_index).T,
            edge_attr=torch.Tensor(edge_attr),
            y=y,
        )
        if k in train:
            all_data[dataset]["train"].append(data)
        else:
            all_data[dataset]["test"].append(data)


  0%|          | 0/15 [00:00<?, ?it/s]

In [3]:
import pickle

pickle.dump(all_data, open("data/tabular/processed_graphs.pkl", "wb"))

In [ ]:
# Configure advanced logging
def setup_logging(timestamp, dataset_name, model_name, experiment_num):
    """Setup comprehensive logging system"""

    log_dir = f"logs/{timestamp}/{dataset_name}/{model_name}/{experiment_num}"
    os.makedirs(log_dir, exist_ok=True)

    logger = logging.getLogger(f"{dataset_name}_{model_name}_{experiment_num}")
    logger.setLevel(logging.DEBUG)

    # File handler
    file_handler = logging.FileHandler(f"{log_dir}/experiment.log")
    file_handler.setLevel(logging.DEBUG)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    # Formatter
    formatter = logging.Formatter(
        "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    )
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # TensorBoard writer
    tb_writer = SummaryWriter(log_dir=log_dir)

    return logger, tb_writer, log_dir


class GNNModel(nn.Module):
    """Enhanced GNN model with toggleable components"""

    def __init__(
        self,
        model_type,
        in_channels,
        hidden_channels,
        out_channels,
        edge_dim,
        num_layers=3,
        heads=4,
        dropout=0.5,
        activation="relu",
        residual=False,
        # ===== TOGGLEABLE COMPONENTS =====
        use_classifier_mlp=False,
        classifier_mlp_dims=[64, 32],
        # =================================
    ):
        super().__init__()
        self.model_type = model_type
        self.layers = nn.ModuleList()
        self.dropout = dropout
        self.residual = residual
        self.activation = self._get_activation(activation)
        self.activation_name = activation

        # === GNN LAYERS ===
        _edge_dim = edge_dim
        self.per_layer_edge_encoders = nn.ModuleList()
        for i in range(num_layers):
            if model_type == "GINE":
                # Edge encoder for this layer
                edge_layer_encoder = nn.Linear(_edge_dim, hidden_channels)
                _edge_dim = hidden_channels
                self.per_layer_edge_encoders.append(edge_layer_encoder)

                # GINE convolution
                gin_nn = nn.Sequential(
                    nn.Linear(
                        in_channels if i == 0 else hidden_channels, hidden_channels
                    ),
                    self._get_activation(activation),
                    nn.Linear(hidden_channels, hidden_channels),
                )
                self.layers.append(GINEConv(gin_nn, edge_dim=hidden_channels))

            elif model_type == "GATv2":
                self.layers.append(
                    GATv2Conv(
                        in_channels if i == 0 else hidden_channels * heads,
                        hidden_channels,
                        heads=heads,
                        edge_dim=edge_dim,
                        dropout=dropout,
                    )
                )
            else:
                raise ValueError(f"Unsupported model type: {model_type}")

        # === CLASSIFIER HEAD ===
        classifier_input_dim = (
            hidden_channels if model_type == "GINE" else hidden_channels * heads
        )

        if use_classifier_mlp:
            classifier_layers = []
            in_dim = classifier_input_dim
            for dim in classifier_mlp_dims:
                classifier_layers.append(nn.Linear(in_dim, dim))
                classifier_layers.append(self._get_activation(activation))
                in_dim = dim
            classifier_layers.append(nn.Linear(dim, out_channels))
            self.classifier = nn.Sequential(*classifier_layers)
        else:
            self.classifier = nn.Linear(classifier_input_dim, out_channels)

    def _get_activation(self, name):
        """Factory method for activation functions"""
        activations = {
            "relu": nn.ReLU(),
            "leaky_relu": nn.LeakyReLU(0.1),
            "elu": nn.ELU(),
            "prelu": nn.PReLU(),
            "selu": nn.SELU(),
            "tanh": nn.Tanh(),
        }
        if name.lower() not in activations:
            raise ValueError(f"Unknown activation: {name}")
        return activations[name.lower()]

    def forward(self, data):
        x, edge_index, edge_attr, batch = (
            data.x,
            data.edge_index,
            data.edge_attr,
            data.batch,
        )

        x_prev = x
        edge_attr_prev = edge_attr

        for i, layer in enumerate(self.layers):
            # GNN layer processing
            if self.model_type == "GINE":
                edge_attr = self.per_layer_edge_encoders[i](edge_attr)
                x = layer(x, edge_index, edge_attr)
            else:
                x = layer(x, edge_index, edge_attr)

            # Activation and residual for nodes
            x = self.activation(x)
            if self.residual and x_prev.shape[1] == x.shape[1]:
                x = x + x_prev

            # Apply intermediate MLP
            x = F.dropout(x, p=self.dropout, training=self.training)

            # Update edge attributes
            if self.residual and edge_attr_prev.shape[1] == edge_attr.shape[1]:
                edge_attr = edge_attr + edge_attr_prev

            # Prepare for next layer
            x_prev = x
            edge_attr_prev = edge_attr

        # Global pooling and classification
        x = global_mean_pool(x, batch)
        return self.classifier(x)


class GNNTrainer:
    """Professional trainer class for GNN experiments"""

    def __init__(self, device="cuda"):
        self.device = torch.device(device)
        self.metric_history = defaultdict(list)

    def train_epoch(self, model, loader, optimizer, criterion):
        """Training loop for one epoch"""
        model.train()
        total_loss = 0
        all_probs, all_labels = [], []

        for batch in loader:
            batch = batch.to(self.device)
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y.long())
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch.num_graphs
            probs = F.softmax(out, dim=1)
            all_probs.append(probs.detach().cpu())
            all_labels.append(batch.y.cpu())

        all_probs = torch.cat(all_probs)
        all_labels = torch.cat(all_labels)
        return total_loss / len(loader.dataset), all_probs, all_labels

    def evaluate(self, model, loader, criterion):
        """Comprehensive model evaluation"""
        model.eval()
        total_loss = 0
        all_probs, all_preds, all_labels = [], [], []

        with torch.no_grad():
            for batch in loader:
                batch = batch.to(self.device)
                out = model(batch)
                loss = criterion(out, batch.y.long())

                total_loss += loss.item() * batch.num_graphs
                probs = F.softmax(out, dim=1)
                preds = probs.argmax(dim=1)

                all_probs.append(probs.cpu())
                all_preds.append(preds.cpu())
                all_labels.append(batch.y.cpu())

        all_probs = torch.cat(all_probs)
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        metrics = self._compute_metrics(all_probs, all_preds, all_labels)
        metrics["loss"] = total_loss / len(loader.dataset)
        return metrics

    def _compute_metrics(self, probs, preds, labels):
        """Calculate comprehensive classification metrics"""
        metrics = {}
        labels_np = labels.numpy()
        probs_np = probs.numpy()
        preds_np = preds.numpy()

        # Basic metrics
        metrics["probs"] = probs_np[:, 1]
        metrics["accuracy"] = (preds_np == labels_np).mean()
        metrics["precision"] = precision_score(
            labels_np, preds_np, average="binary", pos_label=1
        )
        metrics["recall"] = recall_score(
            labels_np, preds_np, average="binary", pos_label=1
        )
        metrics["f1"] = f1_score(labels_np, preds_np, average="binary", pos_label=1)

        # ROC and AUC
        try:
            metrics["roc_auc"] = roc_auc_score(labels_np, probs_np[:, 1])
        except ValueError:
            metrics["roc_auc"] = 0.5

        # Precision-Recall curve
        precision, recall, _ = precision_recall_curve(labels_np, probs_np[:, 1])
        metrics["pr_auc"] = auc(recall, precision)

        # # Confusion matrix
        # metrics["confusion_matrix"] = confusion_matrix(labels_np, preds_np)

        # Classification report
        metrics["classification_report"] = classification_report(
            labels_np, preds_np, output_dict=True
        )

        return metrics

    def train(
        self,
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        scheduler,
        epochs,
        logger,
        tb_writer,
        log_dir,
        patience=128,
    ):
        """Full training loop with early stopping"""
        best_val_f1 = 0
        early_stop_counter = 0
        history = defaultdict(list)

        for epoch in range(1, epochs + 1):
            start_time = time.time()

            # Training
            train_loss, train_probs, train_labels = self.train_epoch(
                model, train_loader, optimizer, criterion
            )
            train_metrics = self._compute_metrics(
                train_probs, train_probs.argmax(dim=1), train_labels
            )

            # Validation
            val_metrics = self.evaluate(model, val_loader, criterion)

            # Update scheduler
            scheduler.step(val_metrics["f1"])

            # Track history
            history["epoch"].append(epoch)
            history["train_loss"].append(train_loss)
            history["train_metrics"].append(train_metrics)
            history["val_loss"].append(val_metrics["loss"])
            history["val_roc_auc"].append(val_metrics["roc_auc"])
            history['val_acc'].append(val_metrics['acc'])
            history["val_f1"].append(val_metrics["f1"])
            history["lr"].append(optimizer.param_groups[0]["lr"])

            # Log metrics
            # epoch_time = time.time() - start_time
            # if epoch % 32 == 0 or epoch == epochs:
            #     logger.info(
            #         f"Epoch {epoch:03d} | Time: {epoch_time:.1f}s | "
            #         f"LR: {history['lr'][-1]:.6f} | "
            #         f"Train Loss: {train_loss:.4f} | "
            #         f"Val Loss: {val_metrics['loss']:.4f} | "
            #         f"Val ROC-AUC: {val_metrics['roc_auc']:.4f} | "
            #         f"Val F1: {val_metrics['f1']:.4f}"
            #     )

            # TensorBoard logging
            tb_writer.add_scalar("Loss/train", train_loss, epoch)
            tb_writer.add_scalar("Loss/val", val_metrics["loss"], epoch)
            tb_writer.add_scalar("ROC-AUC/val", val_metrics["roc_auc"], epoch)
            tb_writer.add_scalar("accuracy/val", val_metrics["acc"], epoch)
            tb_writer.add_scalar("F1/val", val_metrics["f1"], epoch)
            tb_writer.add_scalar("Learning Rate", history["lr"][-1], epoch)

            # Checkpointing
            if val_metrics["f1"] > best_val_f1:
                best_val_f1 = val_metrics["f1"]
                early_stop_counter = 0
                torch.save(
                    {
                        "epoch": epoch,
                        "model_state_dict": model.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "val_f1": best_val_f1,
                    },
                    f"{log_dir}/best_model.pth",
                )
                # logger.info(f"New best model: ROC-AUC = {best_val_roc:.4f}")
            else:
                early_stop_counter += 1
                if early_stop_counter >= patience:
                    # logger.info(f"Early stopping at epoch {epoch}")
                    early_stop_counter = 0
                    break

        # Load best model
        checkpoint = torch.load(f"{log_dir}/best_model.pth")
        model.load_state_dict(checkpoint["model_state_dict"])

        return history, model


def plot_metrics(history, config, log_dir):
    """Create comprehensive metric visualizations"""
    plt.figure(figsize=(12, 8))

    # Loss curves
    plt.subplot(2, 2, 1)
    plt.plot(history["epoch"], history["train_loss"], "b-", label="Train")
    plt.plot(history["epoch"], history["val_loss"], "r-", label="Validation")
    plt.title("Loss Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    # # ROC-AUC
    plt.subplot(2, 2, 2)
    plt.plot(history["epoch"], history["val_roc_auc"], "g-")
    plt.title("Validation ROC-AUC")
    plt.xlabel("Epoch")
    plt.ylabel("ROC-AUC")

    # F1 Score
    plt.subplot(2, 2, 3)
    plt.plot(history["epoch"], history["val_f1"], "m-")
    plt.title("Validation F1 Score")
    plt.xlabel("Epoch")
    plt.ylabel("F1")

    # # Learning Rate
    plt.subplot(2, 2, 4)
    plt.plot(history["epoch"], history["lr"], "c-")
    plt.title("Learning Rate Schedule")
    plt.xlabel("Epoch")
    plt.ylabel("Learning Rate")
    plt.yscale("log")

    plt.tight_layout()
    plt.savefig(f"{log_dir}/training_metrics.png")
    plt.close()

    # Save history to CSV
    history_df = pd.DataFrame(history)
    history_df.to_csv(f"{log_dir}/training_history.csv", index=False)


def run_experiments(dataset):
    """Run comprehensive GNN experiments"""
    # Experiment configuration
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_config = {
        "model_types": ["GATv2", "GINE"],
        "activations": ["leaky_relu", "tanh"],
        "param_grid": {
            # Core GNN parameters
            "learning_rate": [1e-3, 1e-4],
            "hidden_channels": [4, 32],
            "dropout": [0.1, 0.5],
            "num_layers": [2],
            "heads": [1, 5],  # For GATv2
            "residual": [False, True],
            # Classifier head configurations
            "use_classifier_mlp": [False, True],
            "classifier_mlp_dims": [[32], [16, 16]],
        },
    }
    trainer = GNNTrainer(
        device="cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.mps.is_available()
        else "cpu"
    )

    for dataset_name in tqdm(dataset):
        # Data preparation
        train_data = dataset[dataset_name]["train"]
        test_data = dataset[dataset_name]["test"]

        for activation in experiment_config["activations"]:
            for model_type in experiment_config["model_types"]:
                # Hyperparameter tuning with cross-validation
                best_score = -1
                best_params = {}
                current_experument_idx = 0

                # Parameter search with cross-validation
                for params in tqdm(
                    ParameterGrid(experiment_config["param_grid"]),
                    desc=f"experiment: {model_type} with {activation} activation",
                ):
                    # Skip invalid configurations
                    if (
                        model_type == "GINE"
                        and "heads" in params
                        and params["heads"] != 1
                    ) or (
                        params["use_classifier_mlp"] is False
                        and params["classifier_mlp_dims"] != [32]
                    ):
                        continue

                    cv_scores = []
                    skf = StratifiedKFold(n_splits=3)

                    # Get labels for stratification
                    labels = [data.y for data in train_data]

                    for fold, (train_idx, val_idx) in enumerate(
                        skf.split(train_data, labels)
                    ):
                        logger, tb_writer, log_dir = setup_logging(
                            timestamp,
                            dataset,
                            f"{model_type}_{activation}",
                            current_experument_idx,
                        )
                        current_experument_idx += 1
                        # Create data loaders
                        train_loader = DataLoader(
                            [train_data[i] for i in train_idx],
                            batch_size=128,
                            shuffle=True,
                        )
                        val_loader = DataLoader(
                            [train_data[i] for i in val_idx], batch_size=128
                        )

                        # Model initialization
                        model = GNNModel(
                            model_type=model_type,
                            in_channels=1,
                            hidden_channels=params["hidden_channels"],
                            out_channels=2,
                            edge_dim=1,
                            num_layers=params["num_layers"],
                            heads=params.get("heads", 4),
                            dropout=params["dropout"],
                            activation=activation,
                            residual=params["residual"],
                            use_classifier_mlp=params["use_classifier_mlp"],
                            classifier_mlp_dims=params["classifier_mlp_dims"],
                        ).to(trainer.device)

                        # Optimizer and scheduler
                        optimizer = torch.optim.AdamW(
                            model.parameters(),
                            lr=params["learning_rate"],
                            weight_decay=1e-5,
                        )
                        scheduler = ReduceLROnPlateau(
                            optimizer,
                            mode="max",
                            factor=0.5,
                            patience=8,
                        )
                        criterion = nn.CrossEntropyLoss()

                        # Training
                        try:
                            history, model = trainer.train(
                                model,
                                train_loader,
                                val_loader,
                                optimizer,
                                criterion,
                                scheduler,
                                epochs=1024,
                                logger=logger,
                                tb_writer=tb_writer,
                                log_dir=log_dir,
                                patience=32,
                            )

                            # Final validation score
                            val_metrics = trainer.evaluate(model, val_loader, criterion)
                            cv_scores.append(val_metrics["roc_auc"])
                        except Exception as e:
                            logger.error(f"CV fold {fold} failed: {str(e)}")
                            cv_scores.append(0.0)

                    # Average CV score
                    mean_score = np.mean(cv_scores)
                    # logger.info(f"Params: {params} | Mean ROC-AUC: {mean_score:.4f}")

                    if mean_score > best_score:
                        best_score = mean_score
                        best_params = params

                # logger.info(f"Best params: {best_params} | ROC-AUC: {best_score:.4f}")

                # Final training with best params on full data
                train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
                test_loader = DataLoader(test_data, batch_size=128)

                logger, tb_writer, log_dir = setup_logging(
                    timestamp,
                    dataset,
                    f"{model_type}_{activation}",
                    'final',
                )

                model = GNNModel(
                    model_type=model_type,
                    in_channels=1,
                    hidden_channels=best_params["hidden_channels"],
                    out_channels=2,
                    edge_dim=1,
                    num_layers=best_params["num_layers"],
                    heads=best_params.get("heads", 4),
                    dropout=best_params["dropout"],
                    activation=activation,
                    residual=best_params["residual"],
                    use_classifier_mlp=best_params["use_classifier_mlp"],
                    classifier_mlp_dims=best_params["classifier_mlp_dims"],
                ).to(trainer.device)

                optimizer = torch.optim.AdamW(
                    model.parameters(),
                    lr=best_params["learning_rate"],
                    weight_decay=1e-5,
                )
                scheduler = ReduceLROnPlateau(
                    optimizer, mode="max", factor=0.5, patience=8
                )
                criterion = nn.CrossEntropyLoss()

                # Training with full dataset
                history, model = trainer.train(
                    model,
                    train_loader,
                    test_loader,
                    optimizer,
                    criterion,
                    scheduler,
                    epochs=1024,
                    logger=logger,
                    tb_writer=tb_writer,
                    log_dir=log_dir,
                    patience=32,
                )

                # Final evaluation
                test_metrics = trainer.evaluate(model, test_loader, criterion)

                # Log final results
                logger.info(f"\n{'=' * 50}")
                logger.info(f"FINAL RESULTS: {model_type} with {activation}")
                logger.info(f"Test ROC-AUC: {test_metrics['roc_auc']:.4f}")
                logger.info(f"Test F1: {test_metrics['f1']:.4f}")
                logger.info(f"Test Accuracy: {test_metrics['accuracy']:.4f}")
                logger.info(f"Test PR-AUC: {test_metrics['pr_auc']:.4f}")
                logger.info("\nClassification Report:")
                logger.info(test_metrics["classification_report"])
                logger.info("=" * 50)

                # Plot metrics
                plot_metrics(history, best_params, log_dir)

                # Save confusion matrix
                # plt.figure(figsize=(8, 6))
                # sns.heatmap(test_metrics["confusion_matrix"], annot=True, fmt="d")
                # plt.title("Confusion Matrix")
                # plt.savefig(f"{log_dir}/confusion_matrix.png")
                # plt.close()

                # Save ROC curve
                # fpr, tpr, _ = roc_curve(
                #     np.array([x.y for x in test_loader.dataset]),
                #     test_metrics["probs"][:, 1].numpy(),
                # )
                # plt.figure(figsize=(8, 6))
                # plt.plot(fpr, tpr, label=f"ROC (AUC = {test_metrics['roc_auc']:.2f})")
                # plt.plot([0, 1], [0, 1], "k--")
                # plt.xlabel("False Positive Rate")
                # plt.ylabel("True Positive Rate")
                # plt.title("ROC Curve")
                # plt.legend()
                # plt.savefig(f"{log_dir}/roc_curve.png")
                # plt.close()

                # Close TensorBoard writer
                tb_writer.close()

In [ ]:
run_experiments(datasets, all_data)

  0%|          | 0/15 [00:00<?, ?it/s]

experiment: GATv2 with leaky_relu activation:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-23 10:52:47,066 - Ionoshp_GATv2_leaky_relu_final - INFO - 
2025-07-23 10:52:47,067 - Ionoshp_GATv2_leaky_relu_final - INFO - FINAL RESULTS: GATv2 with leaky_relu
2025-07-23 10:52:47,067 - Ionoshp_GATv2_leaky_relu_final - INFO - Test ROC-AUC: 0.9939
2025-07-23 10:52:47,067 - Ionoshp_GATv2_leaky_relu_final - INFO - Test F1: 0.7931
2025-07-23 10:52:47,068 - Ionoshp_GATv2_leaky_relu_final - INFO - Test Accuracy: 0.6620
2025-07-23 10:52:47,068 - Ionoshp_GATv2_leaky_relu_final - INFO - Test PR-AUC: 0.9965
2025-07-23 10:52:47,068 - Ionoshp_GATv2_leaky_relu_final - INFO - 
Classification Report:
2025-07-23 10:52:47,068 - Ionoshp_GATv2_leaky_relu_final - INFO - {'False': {'precision': 1.0, 'recall': 0.04, 'f1-score': 0.07692307692307693, 'support': 25.0}, 'True': {'precision': 0.6571428571428571, 'recall': 1.0, 'f1-score': 0.7931034482758621, 'support': 46.0}, 'accuracy': 0.6619718309859155, 'macro avg': {'precision': 0.8285714285714285, 'recall': 0.52, 'f1-score': 0.4350132625994695, '

experiment: GINE with leaky_relu activation:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-23 11:03:13,754 - Ionoshp_GINE_leaky_relu_final - INFO - 
2025-07-23 11:03:13,754 - Ionoshp_GINE_leaky_relu_final - INFO - FINAL RESULTS: GINE with leaky_relu
2025-07-23 11:03:13,754 - Ionoshp_GINE_leaky_relu_final - INFO - Test ROC-AUC: 0.9852
2025-07-23 11:03:13,754 - Ionoshp_GINE_leaky_relu_final - INFO - Test F1: 0.9783
2025-07-23 11:03:13,755 - Ionoshp_GINE_leaky_relu_final - INFO - Test Accuracy: 0.9718
2025-07-23 11:03:13,755 - Ionoshp_GINE_leaky_relu_final - INFO - Test PR-AUC: 0.9902
2025-07-23 11:03:13,755 - Ionoshp_GINE_leaky_relu_final - INFO - 
Classification Report:
2025-07-23 11:03:13,755 - Ionoshp_GINE_leaky_relu_final - INFO - {'False': {'precision': 0.96, 'recall': 0.96, 'f1-score': 0.96, 'support': 25.0}, 'True': {'precision': 0.9782608695652174, 'recall': 0.9782608695652174, 'f1-score': 0.9782608695652174, 'support': 46.0}, 'accuracy': 0.971830985915493, 'macro avg': {'precision': 0.9691304347826086, 'recall': 0.9691304347826086, 'f1-score': 0.96913043478260

...

experiment: GATv2 with tanh activation:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-23 12:02:32,734 - Cryotherapy_GATv2_tanh_final - INFO - 
2025-07-23 12:02:32,734 - Cryotherapy_GATv2_tanh_final - INFO - FINAL RESULTS: GATv2 with tanh
2025-07-23 12:02:32,734 - Cryotherapy_GATv2_tanh_final - INFO - Test ROC-AUC: 0.9125
2025-07-23 12:02:32,735 - Cryotherapy_GATv2_tanh_final - INFO - Test F1: 0.7143
2025-07-23 12:02:32,735 - Cryotherapy_GATv2_tanh_final - INFO - Test Accuracy: 0.5556
2025-07-23 12:02:32,735 - Cryotherapy_GATv2_tanh_final - INFO - Test PR-AUC: 0.9247
2025-07-23 12:02:32,735 - Cryotherapy_GATv2_tanh_final - INFO - 
Classification Report:
2025-07-23 12:02:32,736 - Cryotherapy_GATv2_tanh_final - INFO - {'False': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 8.0}, 'True': {'precision': 0.5555555555555556, 'recall': 1.0, 'f1-score': 0.7142857142857143, 'support': 10.0}, 'accuracy': 0.5555555555555556, 'macro avg': {'precision': 0.2777777777777778, 'recall': 0.5, 'f1-score': 0.35714285714285715, 'support': 18.0}, 'weighted avg': {'preci

experiment: GINE with tanh activation:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-23 12:05:23,626 - Cryotherapy_GINE_tanh_final - INFO - 
2025-07-23 12:05:23,626 - Cryotherapy_GINE_tanh_final - INFO - FINAL RESULTS: GINE with tanh
2025-07-23 12:05:23,626 - Cryotherapy_GINE_tanh_final - INFO - Test ROC-AUC: 0.9750
2025-07-23 12:05:23,627 - Cryotherapy_GINE_tanh_final - INFO - Test F1: 0.0000
2025-07-23 12:05:23,627 - Cryotherapy_GINE_tanh_final - INFO - Test Accuracy: 0.4444
2025-07-23 12:05:23,627 - Cryotherapy_GINE_tanh_final - INFO - Test PR-AUC: 0.9826
2025-07-23 12:05:23,627 - Cryotherapy_GINE_tanh_final - INFO - 
Classification Report:
2025-07-23 12:05:23,628 - Cryotherapy_GINE_tanh_final - INFO - {'False': {'precision': 0.4444444444444444, 'recall': 1.0, 'f1-score': 0.6153846153846154, 'support': 8.0}, 'True': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0}, 'accuracy': 0.4444444444444444, 'macro avg': {'precision': 0.2222222222222222, 'recall': 0.5, 'f1-score': 0.3076923076923077, 'support': 18.0}, 'weighted avg': {'precision': 0.1

experiment: GATv2 with leaky_relu activation:   0%|          | 0/128 [00:00<?, ?it/s]